In [ ]:
# Si no tenemos el módulo instalado
!pip install google-cloud-secret-manager
!pip install langchain-openai
!pip install langchain
!pip install tiktoken
!pip install cohere

## Memoria - ConversationBufferMemory


In [1]:
#Primero autenticamos el usuario :
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

from google.cloud import secretmanager

# Creamos un Cliente de SecretManager:
client = secretmanager.SecretManagerServiceClient()
secret_name = "openai-token"
project_id = '196177837977'

# Contruimos a F-String:
resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"

# Obtenemos el secreto :
response = client.access_secret_version(request={"name": resource_name})
secret_string = response.payload.data.decode('UTF-8')


# Configurar el motor de OpenAI
engine = "gpt-4"


In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = ChatOpenAI(temperature=0.0, model=engine, openai_api_key=secret_string)
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # Si ponemos verbose=True devuelve datos de la plantilla
)

In [4]:
conversation.predict(input="Hola, mi nombre es Carlos")

'¡Hola, Carlos! Es un placer conocerte. ¿Cómo puedo ayudarte hoy?'

In [5]:
conversation.predict(input="¿Cuál es la capital de Honduras?")

'La capital de Honduras es Tegucigalpa.'

In [6]:
conversation.predict(input="¿Cuál es mi nombre?")

'Tu nombre es Carlos.'

In [7]:
print(memory.buffer)

Human: Hola, mi nombre es Carlos
AI: ¡Hola, Carlos! Es un placer conocerte. ¿Cómo puedo ayudarte hoy?
Human: ¿Cuál es la capital de Honduras?
AI: La capital de Honduras es Tegucigalpa.
Human: ¿Cuál es mi nombre?
AI: Tu nombre es Carlos.


## ConversationBufferWindowMemory

In [12]:
from langchain.memory import ConversationBufferWindowMemory

In [20]:
llm = ChatOpenAI(temperature=0.0, model=engine, openai_api_key=secret_string)
## USamos conversationBufferWindowMemory para almacenar el contexto de la conversación

memory = ConversationBufferWindowMemory(k=1) # Numero de interacciones en AI y usuario

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # Si usas True podrás ver el prompt real que se está pasando al modelo

)

In [21]:
conversation.predict(input="Hola, mi nombre es Carlos")

'¡Hola, Carlos! Es un placer conocerte. ¿Cómo puedo ayudarte hoy?'

In [22]:
conversation.predict(input="¿Cuál es la capital de Honduras?")

'La capital de Honduras es Tegucigalpa.'

In [23]:
conversation.predict(input="¿Cuál es mi nombre") # No lo recordará ya que k=1. Cambia k=2 para ver que funciona

'Tu nombre es Carlos.'

In [39]:
## ConversationSummaryMemory

In [24]:
from langchain.memory import ConversationSummaryBufferMemory

In [25]:
# creamos un texto
schedule = "Hay una reunión a las 8 a. m. con tu equipo de producto. \
Necesitarás tener preparada tu presentación de PowerPoint. \
De 9 a. m. a 12 p. m. tienes tiempo para trabajar en tu proyecto LangChain, \
que avanzará rápidamente porque LangChain es una herramienta muy potente.\
Al mediodía, almuerzo en el restaurante italiano con un cliente que viene conduciendo\
desde más de una hora de distancia para reunirse contigo y entender lo último en IA.\
Asegúrate de traer tu portátil para mostrar la última demo de LLM."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hola"}, {"output": "¿Qué hay?"})
memory.save_context({"input": "No mucho, por aquí..."},
                    {"output": "Guay"})
memory.save_context({"input": "Qué hay en la agenda hoy?"},
                    {"output": f"{schedule}"})

In [26]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI and asks about the day's agenda. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting at an Italian restaurant with a client interested in the latest in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo."}

In [27]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [28]:
conversation.predict(input="Qué hay en la agenda para hoy?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks about the day's agenda. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting at an Italian restaurant with a client interested in the latest in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo.
Human: Qué hay en la agenda para hoy?
AI:

> Finished chain.


'Buenos días, hoy tienes una reunión por la mañana con el equipo de producto. Luego, tendrás tiempo para trabajar en el proyecto LangChain. Al mediodía, tienes una comida en un restaurante italiano con un cliente interesado en lo último en IA. Recuerda llevar tu portátil para mostrar la última demostración de LLM.'